In [1]:
import numpy as np
#import multi_gpu  #use on AWS
import string
import re
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Activation, Dense

/Users/Pan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
N_GPU = 1 # you can experiment with more GPUs, it gets interesting with a high SEQUENCE_LEN
SEQUENCE_LEN = 60
BATCH_SIZE = 512
EPOCHS = 20
HIDDEN_LAYERS_DIM = 512
LAYER_COUNT = 4
DROPOUT = 0.2

# Data cleaning

In [3]:
# read the 2017 data to see the original data structure
temp=pd.read_json("./jsondata/condensed_2017.json")
temp.head()

,created_at,favorite_count,id_str,in_reply_to_user_id_str,is_retweet,retweet_count,source,text
0,2018-01-01 13:37:52,51473,947824196909961216,NaN,False,8237,Twitter for iPhone,Will be leaving Florida for Washington (D.C.) ...
1,2018-01-01 12:44:40,53557,947810806430826496,25073877.0,False,14595,Twitter for iPhone,Iran is failing at every level despite the ter...
2,2018-01-01 12:12:00,138808,947802588174577664,NaN,False,49566,Twitter for iPhone,The United States has foolishly given Pakistan...
3,2017-12-31 23:43:04,154769,947614110082043904,NaN,False,35164,Twitter for iPhone,HAPPY NEW YEAR! We are MAKING AMERICA GREAT AG...
4,2017-12-31 22:18:20,157655,947592785519173632,NaN,False,39428,Twitter for iPhone,As our Country rapidly grows stronger and smar...


In [4]:
file_name="./jsondata/condensed_"
mydf = pd.read_json(file_name+str(2000+9)+".json")
mydf = mydf.loc[mydf['is_retweet']==False][['text']]
    
for year in range(10,19):
    df = pd.read_json(file_name+str(2000+year)+".json")
    df = df.loc[df['is_retweet']==False][['text']]
    mydf=mydf.append(df,ignore_index=True)
mydf.head()

,text
0,From Donald Trump: Wishing everyone a wonderfu...
1,Trump International Tower in Chicago ranked 6t...
2,Wishing you and yours a very Happy and Bountif...
3,Donald Trump Partners with TV1 on New Reality ...
4,"--Work has begun, ahead of schedule, to build ..."


In [28]:
for i in range(len(mydf)):
    # remove url
    mydf['text'][i]= re.sub(r"http\S+", "", mydf['text'][i])
    
    # remove these charcters (" . -) at the beginning of the string 
    # remove (") at the end of the string 
    # remove whitespaces at the beginning and the end
    mydf['text'][i]= mydf['text'][i].strip().lstrip("-").lstrip('"').lstrip(".").rstrip('"').strip()
    
mydf.head()

,text
0,From Donald Trump: Wishing everyone a wonderfu...
1,Trump International Tower in Chicago ranked 6t...
2,Wishing you and yours a very Happy and Bountif...
3,Donald Trump Partners with TV1 on New Reality ...
4,"Work has begun, ahead of schedule, to build th..."


In [6]:
len(mydf) #number of tweets

32858

In [24]:
# create train and test

train=mydf.sample(frac=0.9,random_state=200)
test=mydf.drop(train.index)

In [32]:
# create train and test
text_train=''
text_val=''

for i in train['text']:
    text_train+=i+' '

for i in test['text']:
    text_val+=i+' '

In [33]:
# remove non-English letters & emoji
text_train=text_train.encode('ascii',errors='ignore').decode()
text_val=text_val.encode('ascii',errors='ignore').decode()

chars = sorted(list(set(text_train+text_val)))
','.join(chars) #show remaining letters

'\n,\r, ,!,",#,$,%,&,\',(,),*,+,,,-,.,/,0,1,2,3,4,5,6,7,8,9,:,;,=,?,@,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,[,\\,],_,`,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,{,|,},~'

In [34]:
# remove some more special characters to reduce the size of parameter matrix --> increase training spped
text_train=re.sub("[-+~`*\]\[|\\n\\r\(\)\{\}<>]", "", text_train)
text_train=text_train.replace("/","").replace("\\", "") .replace("=", "").replace('\'',"'")
text_val=re.sub("[-+~`*\]\[|\\n\\r\(\)\{\}<>]", "", text_val)
text_val=text_val.replace("/","").replace("\\", "") .replace("=", "").replace('\'',"'")

chars = sorted(list(set(text_train+text_val)))
','.join(chars)#show remaining letters

' ,!,",#,$,%,&,\',,,.,0,1,2,3,4,5,6,7,8,9,:,;,?,@,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,_,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z'

In [35]:
text_train_len = len(text_train)
text_val_len = len(text_val)
print("Total of %d characters" % (text_train_len + text_val_len))

Total of 3458381 characters


In [37]:
# generic vocabulary
VOCABULARY_SIZE = len(chars)
characters_to_ix = {c:i for i,c in enumerate(chars)}
print("vocabulary len = %d" % VOCABULARY_SIZE)
print(chars)

vocabulary len = 77
[' ', '!', '"', '#', '$', '%', '&', "'", ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [38]:
text_train

'@A_Beil   Thanks Andrew! Thank you Greensboro, North Carolina! Will be back soon! #AmericaFirst @Mr_president706: @realDonaldTrump please run for president, and show these so called Republicans how to get the job done. RT @CPACnews: ACU Announces @realDonaldTrump will be a featured speaker at #CPAC2013!  Get tickets today at @SAVEHICNO1: MR. TRUMP, I THINK YOU OUGHT TO DO AN APPRENTICE SHOW FOR PEOPLE TO COME UP FOR A WINNING STRADEGY FOR YOU TO RUN IN 2016 @HomesWestfield @EricTrump @MyFoxNY @rosannascotto @StJude  Thank you. @MotivationIdeas: Sometimes by losing a battle you find a new way to win the war.  Donald Trump The leader and negotiators representing Mexico are far smarter and more cunning than the leader and negotiators representing the U.S.! @rdowns I never went bankrupt. Enemies love to say I did. Didn\'t happen. @manc999 Write to Mark Burnett. I love Mexico but not the unfair trade deals that the US so stupidly makes with them. Really bad for US jobs, only good for Mexic

In [16]:
def describe_batch(X, y, samples=3):
    for i in range(samples):
        sentence = ""
        for s in range(SEQUENCE_LEN):
            sentence += chars[X[i,s,:].argmax()]
        next_char = chars[y[i,:].argmax()]
        
        print("sample #%d: ...%s -> '%s'" % (
            i,
            sentence[-20:],
            next_char
        ))

def batch_generator(text, count):
    """Generate batches for training"""
    while True: # keras wants that for reasons
        for batch_ix in range(count):
            X = np.zeros((BATCH_SIZE, SEQUENCE_LEN, VOCABULARY_SIZE))
            y = np.zeros((BATCH_SIZE, VOCABULARY_SIZE))

            batch_offset = BATCH_SIZE * batch_ix

            for sample_ix in range(BATCH_SIZE):
                sample_start = batch_offset + sample_ix
                for s in range(SEQUENCE_LEN):
                    X[sample_ix, s, characters_to_ix[text[sample_start+s]]] = 1
                y[sample_ix, characters_to_ix[text[sample_start+s+1]]]=1

            yield X, y


In [17]:
def build_model(gpu_count=1):
    """Build a Keras sequential model for training the char-rnn"""
    model = Sequential()
    for i in range(LAYER_COUNT):
        model.add(
            LSTM(
                HIDDEN_LAYERS_DIM, 
                return_sequences=True if (i!=(LAYER_COUNT-1)) else False,
                input_shape=(SEQUENCE_LEN, VOCABULARY_SIZE),
            )
        )
        model.add(Dropout(DROPOUT))
    
    model.add(Dense(VOCABULARY_SIZE))
    model.add(Activation('softmax'))
    
    if gpu_count>1:
        model = multi_gpu.make_parallel(model, gpu_count)
    
    model.compile(loss='categorical_crossentropy', optimizer="adam")
    return model

In [18]:
for ix, (X,y) in enumerate(batch_generator(text_train, count=1)):
    # describe some samples from the first batch
    describe_batch(X, y, samples=5)
    break

sample #0: ...nderful holiday & a  -> 'h'
sample #1: ...derful holiday & a h -> 'a'
sample #2: ...erful holiday & a ha -> 'p'
sample #3: ...rful holiday & a hap -> 'p'
sample #4: ...ful holiday & a happ -> 'y'


### Training

In [21]:
training_model = build_model(gpu_count=N_GPU)

train_batch_count = (text_train_len - SEQUENCE_LEN) // BATCH_SIZE
val_batch_count = (text_val_len - SEQUENCE_LEN) // BATCH_SIZE
print("training batch count: %d" % train_batch_count)
print("validation batch count: %d" % val_batch_count)

# checkpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping
filepath = "./%d-gpu_BS-%d_%d-%s_dp%.2f_%dS_epoch{epoch:02d}-loss{loss:.4f}-val-loss{val_loss:.4f}_weights" % (
    N_GPU,
    BATCH_SIZE,
    LAYER_COUNT,
    HIDDEN_LAYERS_DIM,
    DROPOUT,
    SEQUENCE_LEN
)
checkpoint = ModelCheckpoint(
    filepath,
    save_weights_only=True
)
# early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks_list = [checkpoint, early_stopping]

training batch count: 5949
validation batch count: 805


In [22]:
history = training_model.fit_generator(
    batch_generator(text_train, count=train_batch_count),
    train_batch_count,
    max_queue_size=1, # no more than one queued batch in RAM
    epochs=EPOCHS,
    callbacks=callbacks_list,
    validation_data=batch_generator(text_val, count=val_batch_count),
    validation_steps=val_batch_count,
    initial_epoch=0
)

Epoch 1/20
   6/5949 [..............................] - ETA: 114:30:11 - loss: 4.1929

KeyboardInterrupt: 